In [1]:
import pickle
import sys
import tensorflow as tf
from tqdm import tqdm

In [11]:
def get_labels():
    with open("/Users/pkaur/tensorflow/SignLanguage/InceptionModel/retrained_labels.txt", 'r') as fin:
        labels = [line.rstrip('\n') for line in fin]
    return labels

In [12]:
"""
Go through all of our images and save out the pool layer
representation of our images. This is not a prediction, but rather the
convolutional representation of features that we can then pass to an
RNN. The idea is that later we'll add an RNN layer directly onto
the CNN. This gives us a way to test if that's a good strategy
before putting in the work required to do so.
"""

def predict_on_frames(frames,labels,image_path):
    # Unpersists graph from file
    with tf.gfile.FastGFile("/Users/pkaur/tensorflow/SignLanguage/InceptionModel/retrained_graph.pb", 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

    with tf.Session() as sess:
        pool_tensor = sess.graph.get_tensor_by_name('pool_3:0')

        # Moving this into the session to make it faster.
        cnn_features = []
        pbar = tqdm(total=len(frames))
        for i, frame in enumerate(frames):
            filename = frame[0]
            label = frame[1]

            # For some reason, we have a tar reference in our pickled file.
            if 'tar' in filename:
                continue

            # Get the image path.
            image = image_path + label + "/" + filename

            # Read in the image_data
            image_data = tf.gfile.FastGFile(image, 'rb').read()

            try:
                cnn_representation = sess.run(pool_tensor,{'DecodeJpeg/contents:0': image_data})
            except KeyboardInterrupt:
                print("You quit with ctrl+c")
                sys.exit()
            except:
                print("Error making prediction, continuing.")
                continue

            # Save it out.
            frame_row = [cnn_representation, label]
            cnn_features.append(frame_row)

            if i > 0 and i % 10 == 0:
                pbar.update(10)

        pbar.close()

        return cnn_features

In [17]:
 def create_cnn_train(labels):
    with open('/Users/pkaur/tensorflow/SignLanguage/labeled-images.pkl', 'rb') as fin:
        frames = pickle.load(fin)

    image_path = '/Users/pkaur/tensorflow/SignLanguage/SignDataTrain/'
    train_cnn_features = predict_on_frames(frames, labels, image_path)

    # Save the features
    with open('/Users/pkaur/tensorflow/SignLanguage/cnn-features.pkl', 'wb') as fout:
        pickle.dump(train_cnn_features, fout)

    print("Done on Train.")
    

def create_cnn_test(labels):
    
    with open('/Users/pkaur/tensorflow/SignLanguage/test-data-labeled-images.pkl', 'rb') as fin:
        frames = pickle.load(fin)
    
    image_path = '/Users/pkaur/tensorflow/SignLanguage/SignDataTest/TestSet4/'
    test_cnn_features = predict_on_frames(frames,labels,image_path)
    
    # Save the features
    with open('/Users/pkaur/tensorflow/SignLanguage/test-data-cnn-features.pkl', 'wb') as fout:
        pickle.dump(test_cnn_features, fout)
    
    print("Done on Test.")

    
def main():
    labels = get_labels()
    
    #create_cnn_train(labels);
    create_cnn_test(labels);

In [18]:
if __name__ == '__main__':
    main()



  0%|          | 0/222 [00:00<?, ?it/s]

  5%|▍         | 10/222 [00:33<11:38,  3.30s/it]

  9%|▉         | 20/222 [00:37<08:15,  2.45s/it]

 14%|█▎        | 30/222 [00:42<05:57,  1.86s/it]

 18%|█▊        | 40/222 [00:47<04:23,  1.45s/it]

 23%|██▎       | 50/222 [00:52<03:20,  1.16s/it]

 27%|██▋       | 60/222 [00:57<02:37,  1.03it/s]

 32%|███▏      | 70/222 [01:03<02:09,  1.17it/s]

 36%|███▌      | 80/222 [01:08<01:47,  1.32it/s]

 41%|████      | 90/222 [01:13<01:29,  1.47it/s]

 45%|████▌     | 100/222 [01:18<01:15,  1.62it/s]

 50%|████▉     | 110/222 [01:23<01:04,  1.75it/s]

 54%|█████▍    | 120/222 [01:27<00:55,  1.85it/s]

 59%|█████▊    | 130/222 [01:32<00:47,  1.93it/s]

 63%|██████▎   | 140/222 [01:37<00:41,  1.97it/s]

 68%|██████▊   | 150/222 [01:42<00:35,  2.00it/s]

 72%|███████▏  | 160/222 [01:46<00:30,  2.04it/s]

 77%|███████▋  | 170/222 [01:51<00:25,  2.03it/s]

 81%|████████  | 180/222 [01:56<00:20,  2.06it/s]

 86%|████████▌ | 190/222 [02:01<00:15,  2.08it/s

Done on Test.
